In [33]:
import os
import numbers

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid, save_image

print('PyTorch version:', torch.__version__)
print('torchvision version:', torchvision.__version__)
can_use_gpu = torch.cuda.is_available()
print('Is GPU available:', use_gpu)

PyTorch version: 1.0.0
torchvision version: 0.2.1
Is GPU available: True


In [39]:
# general settings

device = torch.device('cuda' if can_use_gpu else 'cpu')

batchsize = 5

height_for_cropping = 128
width_for_cropping = 128

# warning:cuDNN's randomness is still remaining
seed = 1
torch.manual_seed(seed)

# TODO:change directory names for GPU machine's file system
# directory settings
root_dir = '../hoge/'

# training data directory
image_dir = root_dir + 'hoge/'
label_dir = root_dir + 'hoge/'

# directory to save model output
estimated_label_dir = root_dir + 'hoge/'
if not os.path.exists(estimated_label_dir):
    os.mkdir(estimated_label_dir)

# directory to save model weights and training log
log_dir = root_dir + 'hoge/'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

FileNotFoundError: [Errno 2] No such file or directory: '../hoge/hoge/'

In [40]:
class DocDataset(Dataset):
    def __init__(self, image_dir, label_dir, file_name_list,
                 transform_sync=None, transform_image=None, transform_label=None):
        assert(image_dir[-1] == '/')
        assert(label_dir[-1] == '/')
        self.image_dir = image_dir
        self.label_dir = label_dir
        
        # image or label filename list in image_dir or label_dir (to speedup train_test_split, I'll split file name list)
        # I expect corresponding image and label have same filename
        # This sort is so that following __getitem__ method expect file_name_list have unique order
        self.file_name_list = sorted(file_name_list) 
        
        # to do same random cropping for corresponding image and label
        self.transform_sync = transform_sync
        self.transform_image = transform_image
        self.transform_label = transform_label
        
    def __len__(self):
        return len(self.file_name_list)
    
    def __getitem__(self, idx):
        image_name = self.image_dir + self.file_name_list[idx]
        label_name = self.label_dir + self.file_name_list[idx]
        
        image = Image.open(image_name)
        label = Image.open(label_name)
        
        if self.transform_sync is not None:
            image, label = self.transform_sync(image, label)
        if self.transform_image is not None:
            image = self.transform_image(image)
        if self.transform_label is not None:
            label = self.transform_label(label) 
            
        return image, label

In [41]:
# split to train data and validation data for simplicity
# TODO:test should be conducted by isolated test data (different document)

# sort to eliminate os.listdir randomness
# I expect corresponding image and label have same filename
file_name = sorted(os.listdir(image_dir))
train_file_name, validation_file_name = train_test_split(file_name, test_size=0.2, random_state=seed)

print('The number of training data:', len(train_file_name))
print('The number of validation data:', len(validation_file_name))

FileNotFoundError: [Errno 2] No such file or directory: '../hoge/hoge/'

In [42]:
# transform for synchronize cropping for image and label
# warning:this class can't do padding
class RandomCropSync(object):
    def __init__(self, size):
        if isinstance(self, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size
    
    def get_params(img, output_size):
        w, h = img.size
        th, tw = output_size
        if w == tw and h == th:
            return 0, 0, h, w
        
        i = random.randint(0, h - th)
        j = random.randint(0, w - tw)
        return i, j, th, tw
    
    def __call__(self, img1, img2):
        assert(img1.size == img2.size)
        i, j, h, w = self.get_params(img1, self.size)
        
        img1_cropped = F.crop(img1, i, j, h, w)
        img2_cropped = F.crop(img2, i, j, h, w)
        
        return img1_cropped, img2_cropped

In [48]:
tf_sync_train = RandomCropSync((height_for_cropping, width_for_cropping)) # use only training
tf_image = transforms.ToTensor() # use always
tf_label = transforms.ToTensor() # use always

train_dataset = DocDataset(image_dir, label_dir, train_file_name,
                           tf_sync_train, tf_image, tf_label)
validation_dataset = DocDataset(image_dir, label_dir, validation_file_name,
                                None, tf_image, tf_label)

train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
# In validation, I'll save estimated label, therefore shuffle=True to save result for different input
validation_loader = DataLoader(validation_dataset, batch_size=batchsize, shuffle=True)

NameError: name 'train_file_name' is not defined

In [26]:
# make dataset
imgDataset = MyDataset(image_dir, label_dir, transform_image = tf_image, transform_label = tf_label)

# split to train data and validation data
train_data, validation_data = train_test_split(imgDataset, test_size = 0.2, random_state = seed)

print('The number of training data:', len(train_data))
print('The number of validation data:', len(validation_data))


# In[6]:


# make DataLoader
train_loader = DataLoader(train_data, batch_size = batchsize, shuffle = True)
validation_loader = DataLoader(validation_data, batch_size = batchsize, shuffle = True)

RandomCrop(size=(5, 5), padding=0)